In [1]:
import time, random
import numpy as np
from gensim.models import word2vec

from sklearn.neighbors import KDTree

sentences = word2vec.Text8Corpus('C:/Users/Akshat/Downloads/text8/text8') # initializing the corpus into the memory
model = word2vec.Word2Vec(sentences, size=200, workers=8)
model.init_sims(replace=True)    # normalize the vectors



C:\Users\Akshat\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#(model.wv.vocab(),100)
words = random.sample(model.wv.vocab.keys(),100)

In [3]:
class ANNSearch:
    word2idx = {}
    idx2word = {}
    data = []

    def __init__(self, model):
        for counter, key in enumerate(model.wv.vocab.keys()):
            self.data.append(model[key])
            self.word2idx[key] = counter
            self.idx2word[counter] = key

        # leaf_size is a hyperparameter
        self.data = np.array(self.data)
        #self.tree=NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(self.data)
        self.tree = KDTree(self.data, leaf_size=100)
        #NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(b)
        
    def search_by_vector(self, v, k=10):
        dists, inds = self.tree.query([v], k)
        return (dists[0], [self.idx2word[idx] for idx in inds[0]])

    def search(self, query, k=10):
        vector = self.data[self.word2idx[query]]
        return self.search_by_vector(vector, k)

In [4]:
# Linear Search
start = time.time()
for word in words:
    model.most_similar(word, topn=10)
stop = time.time()
print("time/query by (gensim's) Linear Search = %.2f ms" % (1000*float(stop-start)/len(words)))

# KDTree Search
search_model = ANNSearch(model)

start = time.time()
for word in words:
    search_model.search(word, k=10)
stop = time.time()
print("time/query by KDTree Search = %.2f ms" % (1000*float(stop-start)/len(words)))

time/query by (gensim's) Linear Search = 7.53 ms
time/query by KDTree Search = 28.90 ms


# Testing of Word2vec 

In [6]:
model.most_similar(positive=['king'],topn=10)

[('queen', 0.6789337396621704),
 ('prince', 0.67159503698349),
 ('kings', 0.6612266302108765),
 ('throne', 0.6527180075645447),
 ('emperor', 0.6159657835960388),
 ('vii', 0.6141725778579712),
 ('aragon', 0.6073325872421265),
 ('pharaoh', 0.6024001240730286),
 ('antiochus', 0.5994974374771118),
 ('judah', 0.595747172832489)]

In [5]:
model.most_similar(positive=["king", "woman"], negative=["man"],topn=10)

[('queen', 0.6284703016281128),
 ('elizabeth', 0.5567423105239868),
 ('prince', 0.5491981506347656),
 ('princess', 0.5454338788986206),
 ('daughter', 0.5422555208206177),
 ('son', 0.5383399724960327),
 ('kings', 0.5379177331924438),
 ('throne', 0.5377295613288879),
 ('empress', 0.5331630110740662),
 ('pharaoh', 0.5235458612442017)]

In [26]:
model.most_similar(positive=["burrito", "chinese"], negative=["mexican"])

[('saucers', 0.5281133055686951),
 ('handakuten', 0.5269892811775208),
 ('furigana', 0.5177161693572998),
 ('animations', 0.5109993815422058),
 ('readymades', 0.5107326507568359),
 ('undeciphered', 0.508619487285614),
 ('logographic', 0.5051226615905762),
 ('chainmail', 0.5051131844520569),
 ('knotted', 0.5044553279876709),
 ('photorealistic', 0.5043172240257263)]

# Testing of KDD Tree algorithm

In [136]:
#a=search_model.search("king",k=10)
a=search_model.search("king",k=10) # identifying the top 10 words closest to the word "king"
b=pd.DataFrame(np.vstack(a).T,columns=['dist','word'])
b=b.sort_values(by=["dist"],ascending=False)
b.head(10)

,dist,word
9,0.8965818068122031,duke
8,0.8831202030205509,elector
7,0.8800078612643552,regent
6,0.8548646067693157,vii
5,0.84263860929568,emperor
4,0.835833653538408,prince
3,0.831681865794242,queen
2,0.8253574871380506,throne
1,0.8241146960812941,kings
0,0.0,king


# Implementing Nearest Neighbour Algorithm

In [93]:
class ANNSearch:
    word2idx = {}
    idx2word = {}
    data = []

    def __init__(self, model):
        for counter, key in enumerate(model.wv.vocab.keys()):
            self.data.append(model[key])
            self.word2idx[key] = counter
            self.idx2word[counter] = key

        # leaf_size is a hyperparameter
        self.data = np.array(self.data)
        self.tree=NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(self.data)
        
    def search_by_vector_NN(self, v, k=3):
        dists, inds = self.tree.radius_neighbors([v], k)
        return (dists[0], [self.idx2word[idx] for idx in inds[0]])

    def search_NN(self, radius_neighbors, k=3):
        vector = self.data[self.word2idx[radius_neighbors]]
        return self.search_by_vector_NN(vector, k)

In [94]:
# NearestNeighbour Search
search_model = ANNSearch(model)

start = time.time()
for word in words:
    search_model.search_NN(word, k=3)
stop = time.time()
print("time/query by NearestNeighbour Search = %.2f ms" % (1000*float(stop-start)/len(words)))

time/query by NearestNeighbour Search = 67.92 ms


In [119]:
a=search_model.search_NN("king",k=1) # searching the word "king" with radius of 1
b=pd.DataFrame(np.vstack(a).T,columns=['dist','word'])
b.head()

,dist,word
0,0.9817682219066718,anjou
1,0.9873020348396145,adolphus
2,0.9002145339624846,pope
3,0.9997684287292153,vi
4,0.9112023601387471,viii


In [125]:
#b.loc[b.dist==max(b.dist),]
b.sort_values(by=["dist"],ascending=False)
b.head(10)

,dist,word
0,0.9817682219066718,anjou
1,0.9873020348396145,adolphus
2,0.9002145339624846,pope
3,0.9997684287292153,vi
4,0.9112023601387471,viii
5,0.9984750279518773,jeroboam
6,0.9824305728046298,tyrant
7,0.9969114910246701,grandson
8,0.9929985222369688,menelik
9,0.9446761171339255,empress
